# 0.0. IMPORTS

In [1]:
import re
import pickle

import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
import xgboost           as xgb
import lightgbm          as lgb

from sklearn.preprocessing   import MinMaxScaler
from sklearn.ensemble        import RandomForestRegressor
from sklearn.metrics         import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing   import LabelEncoder
from sklearn.linear_model    import LinearRegression, Lasso
from category_encoders       import TargetEncoder
from sklearn.preprocessing   import OneHotEncoder
from category_encoders.count import CountEncoder


from sklearn               import model_selection   as ms
from sklearn               import ensemble          as en

f:\SamuelOliveiraAlvesd\Anaconda3\envs\Health_Insurance_Cross_Sell\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## 0.1. Aux Functions

In [2]:
def settings():
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25,12]
    plt.rcParams['font.size'] = 24
    plt.rcParams['figure.dpi'] = 100
    sns.set()


def ml_error( model_name, ytest, yhat ):
    mae = mean_absolute_error( ytest, yhat )
    mape = mean_absolute_percentage_error( ytest, yhat )
    rmse = np.sqrt( mean_squared_error( ytest, yhat ) )
    
    return pd.DataFrame( {'Model name': model_name,
                          'MAE': mae,
                          'MAPE': mape,
                          'RMSE': rmse }, index=[0] )

In [3]:
settings()

## 0.2. Reading Data

In [4]:
df_raw=pd.read_csv('work/treino.csv')
df_test=pd.read_csv('work/teste.csv')

# 1.0. DESCRIÇÃO DOS DADOS

In [5]:
df1=df_raw.copy()

## 1.1. Dimensão dos Dados

In [6]:
df1.shape

(39446, 29)

## 1.2. NA Check

In [7]:
df1.isna().sum()

ID                                                                0
num_fotos                                                       237
marca                                                             0
modelo                                                            0
versao                                                            0
ano_de_fabricacao                                                 0
ano_modelo                                                        0
odometro                                                          0
cambio                                                            0
num_portas                                                        0
tipo                                                              0
blindado                                                          0
cor                                                               0
tipo_vendedor                                                     0
cidade_vendedor                                 

### 1.2.1. Fillout NA

In [8]:
# # num_fotos - considerar nan como 0 fotos > ordinal encoding
df1['num_fotos']=df1['num_fotos'].fillna(0)

## 1.3. Tipos dos Dados

In [9]:
df1.dtypes

ID                                                             object
num_fotos                                                     float64
marca                                                          object
modelo                                                         object
versao                                                         object
ano_de_fabricacao                                               int64
ano_modelo                                                    float64
odometro                                                      float64
cambio                                                         object
num_portas                                                      int64
tipo                                                           object
blindado                                                       object
cor                                                            object
tipo_vendedor                                                  object
cidade_vendedor     

### 1.3.1. Change data types

In [10]:
df1['num_fotos']=df1['num_fotos'].astype(int)
df1['ano_modelo']=df1['ano_modelo'].astype(int)

# 2.0. FEATURE ENGINEERING

In [11]:
df2 = df1.copy()

In [12]:
# ano de fabricacao one hot
bin1 = [1985, 2000]
bin2 = [2000, 2005]
bin3 = [2005, 2010]
bin4 = [2010, 2015]
bin5 = [2015, 2020]
bin6 = [2020, 2025]
bins = [bin1, bin2, bin3, bin4, bin5, bin6]

year_bins = []

for i in range(df2.shape[0]):
    tmp = [0 for i in range(len(bins))]
    ye = df2['ano_de_fabricacao'][i]
    for j in range(len(bins)):
        if ye >= bins[j][0] and ye < bins[j][1]:
            tmp[j] = 1
    year_bins.append(tmp)
year_bins = np.asarray(year_bins)
new_df = pd.DataFrame(year_bins, columns=['Year_Bin'+str(i+1) for i in range(len(bins))])
df2 = pd.concat( [df2, new_df], axis=1 )
# df2 = df2.drop(columns=['ano_de_fabricacao'])

# categoria marca
popular_baixo_padrao = ['FIAT','SUZUKI','CHEVROLET','SMART','HYUNDAI','LIFAN','SSANGYONG','RENAULT','DODGE','ALFA ROMEO',
                        'CITROËN','CHRYSLER','BRM','EFFA']

popular_alto_padrao = ['JEEP','SUBARU','FORD','KIA','CHERY','PEUGEOT','VOLKSWAGEN','NISSAN','JAC','HONDA','MITSUBISHI']

luxo = ['VOLVO','LEXUS','MERCEDES-BENZ','FERRARI','AUDI','TOYOTA','IVECO','MINI','TROLLER']

superluxo = ['PORSCHE','RAM','LAMBORGHINI','JAGUAR','LAND ROVER','MASERATI','BMW']

df2['categoria_marca'] = df2['marca'].apply( lambda x: 'popular_baixo_padrao' if x in popular_baixo_padrao else
                                                       'popular_alto_padrao' if x in popular_alto_padrao else
                                                       'luxo' if x in luxo else
                                                       'superluxo' if x in superluxo 
                                                        else 'nao_identificado' )
# cilindrada
cilindradas = []

for i in range(len(df2)):
    try:
        cilindrada = re.search( "[0-9]{1}.[0-9]{1}", df2['versao'][i] )[0]
    except:
        cilindrada = "desconhecida"
        
    cilindradas.append( cilindrada )
    
df_cilindradas = pd.DataFrame (cilindradas, columns = ['cilindrada'])
df2 = pd.concat( [df2,df_cilindradas],axis=1)

# turbo
turbo_list = []

for i in range(len(df2)):
    try:
        turbo = re.search( "TURBO", df2['versao'][i] )[0]
    except:
        turbo = "NÃO TURBO"
        
    turbo_list.append( turbo )
    
df_turbo = pd.DataFrame (turbo_list, columns = ['turbo'])
df2 = pd.concat( [df2,df_turbo],axis=1)

# 4x4
offroad_list = []

for i in range(len(df2)):
    try:
        offroad = re.search( "4X4", df2['versao'][i] )[0]
    except:
        offroad = "NÃO 4x4"
        
    offroad_list.append( offroad )
    
df_offroad = pd.DataFrame (offroad_list, columns = ['offroad'])
df2 = pd.concat( [df2,df_offroad],axis=1)

# combustivel
df2['combustivel'] = df2['versao'].apply( lambda x: re.search( "GASOLINA", x )[0] if re.search( "GASOLINA", x ) is not None else 
                                                    re.search( "FLEX", x )[0] if re.search( "FLEX", x ) is not None else
                                                    re.search( "HYBRID", x )[0] if re.search( "HYBRID", x ) is not None else
                                                    re.search( "DIESEL", x )[0] if re.search( "DIESEL", x ) is not None else
                                                    re.search( "ELECTIRC", x )[0] if re.search( "ELECTIRC", x ) is not None else
                                                    re.search( "ELÉTRICO", x )[0] if re.search( "ELÉTRICO", x ) is not None else
                                                    re.search( "HÍBRIDO", x )[0] if re.search( "HÍBRIDO", x ) is not None else
                                                    re.search( "GÁS", x )[0] if re.search( "GÁS", x ) is not None else "DESCONHECIDO")

In [13]:
df2['cilindrada'] = df2['cilindrada'].apply( lambda x: "0" if x == "desconhecida" else x )
df2['cilindrada'] = df2['cilindrada'].astype(float)
# df2['faixa_cilindrada'] = df2['cilindrada'].apply( lambda x: "ate_1.0" if x <= 1.0 else
#                                                 "ate_1.6" if x <= 1.6 else
#                                                 "ate_2.0" if x <= 2.0 else
#                                                 "ate_2.5" if x<=2.5 else
#                                                 "ate_3.0" if x<=3.0 else
#                                                 "ate_5.0" if x<=5.0 else
#                                                 "mais_de_5.0" if x >5.0 else "desconhecido" )
# df2 = df2.drop(columns=['cilindrada'])

In [14]:
df2['categoria_marca'].unique()

array(['popular_alto_padrao', 'superluxo', 'popular_baixo_padrao', 'luxo'],
      dtype=object)

# 3.0. FILTRAGEM DE VARIÁVEIS

In [15]:
df3 = df2.copy()

## 3.1. Filtragem das Linhas

In [16]:
# # substituindo ano de fabricacao caso a diferença para o ano modelo seja superior a 1 ano
# df3['dif_ano'] = df3.apply(lambda x: x['ano_modelo'] - x['ano_de_fabricacao'], axis=1)
# df3[df3['dif_ano'] > 1].count()

# df3['ano_de_fabricacao'] = df3.apply(lambda x: (x['ano_modelo']-1) if x['dif_ano'] > 1 else x['ano_de_fabricacao'], axis=1)

## 3.2. Seleção das Colunas

In [17]:
# df3 = df3.drop( columns=['elegivel_revisao','attr_veiculo_alienado'])

# 4.0. EDA

In [18]:
df4 = df3.copy()

# 5.0. DATA PREPARATION

In [19]:
df5 = df4.copy()

## 5.1. Encoding

In [20]:
# colunas pra dropar: id
df5 = df5.drop( columns=['ID','ano_de_fabricacao'] )

# binario (0/1): blindado, troca, elegivel_revisao, attr_veiculo_aceita_troca, attr_veiculo_único_dono,
# attr_veiculo_todas_as_revisões_feitas_pela_concessionária, attr_veiculo_ipva_pago, attr_veiculo_licenciado
# attr_veiculo_ipva_pago, attr_veiculo_licenciado, attr_veiculo_garantia_de_fábrica
df5['offroad'] = df5['offroad'].apply( lambda x: 1 if x == '4X4' else 0 )
df5['turbo'] = df5['turbo'].apply( lambda x: 1 if x == 'TURBO' else 0 )
df5['blindado'] = df5['blindado'].apply( lambda x: 0 if x == "false" else 1 if x=="true" else 0 )
df5['troca'] = df5['troca'].apply( lambda x: 0 if x == "false" else 1 if x=="true" else 0 )
#df5['elegivel_revisao'] = df5['elegivel_revisao'].apply( lambda x: 0 if x == "false" else 1 if x=="true" else 0 )
# df5['entrega_delivery'] = df5['entrega_delivery'].apply( lambda x: 0 if x == "false" else 1 if x=="true" else 0 )
# df5['attr_veiculo_aceita_troca'] = df5['attr_veiculo_aceita_troca'].apply( lambda x: 0 if x == "Aceita troca" else 0 )
# df5['attr_veiculo_único_dono'] = df5['attr_veiculo_único_dono'].apply( lambda x: 0 if x == "Único dono" else 0 )
# df5['attr_veiculo_todas_as_revisões_feitas_pela_concessionária'] = df5['attr_veiculo_todas_as_revisões_feitas_pela_concessionária'].apply( lambda x: 0 if x == "Todas as revisões feitas pela concessionária" else 0 )
# df5['attr_veiculo_todas_as_revisões_feitas_pela_agenda_do_carro'] = df5['attr_veiculo_todas_as_revisões_feitas_pela_agenda_do_carro'].apply( lambda x: 0 if x == "Todas as revisões feitas pela agenda do carro" else 0 )
# df5['attr_veiculo_ipva_pago'] = df5['attr_veiculo_ipva_pago'].apply( lambda x: 0 if x == "IPVA pago" else 0 )
# df5['attr_veiculo_licenciado'] = df5['attr_veiculo_licenciado'].apply( lambda x: 0 if x == "Licenciado" else 0 )
# df5['attr_veiculo_garantia_de_fábrica'] = df5['attr_veiculo_garantia_de_fábrica'].apply( lambda x: 0 if x == "Garantia de fábrica" else 0 )
df5 = df5.drop( columns = ['attr_veiculo_aceita_troca','attr_veiculo_único_dono','attr_veiculo_todas_as_revisões_feitas_pela_concessionária',
                           'attr_veiculo_todas_as_revisões_feitas_pela_agenda_do_carro','attr_veiculo_ipva_pago','attr_veiculo_licenciado',
                           'attr_veiculo_garantia_de_fábrica','attr_veiculo_alienado','elegivel_revisao'] )

# dummies: marca, cambio, tipo, tipo_vendedor
encoder = OneHotEncoder(handle_unknown='ignore')

df_cambio = pd.DataFrame(encoder.fit_transform(df5[['cambio']]).toarray())
df_cambio.columns = encoder.get_feature_names_out()
df5 = df5.join(df_cambio)
pickle.dump( encoder, open( 'cambio_encoding', 'wb' ) )

df_tipo_vendedor = pd.DataFrame(encoder.fit_transform(df5[['tipo_vendedor']]).toarray())
df_tipo_vendedor.columns = encoder.get_feature_names_out()
df5 = df5.join(df_tipo_vendedor)
# df5['tipo_vendedor'] = ohe.fit_transform(df5[['tipo_vendedor']])
pickle.dump( encoder, open( 'tipo_vendedor_encoding', 'wb' ) )

df_tipo_anuncio = pd.DataFrame(encoder.fit_transform(df5[['tipo_anuncio']]).toarray())
df_tipo_anuncio.columns = encoder.get_feature_names_out()
df5 = df5.join(df_tipo_anuncio)
# df5['tipo_anuncio'] = ohe.fit_transform(df5[['tipo_anuncio']])
pickle.dump( encoder, open( 'tipo_anuncio_encoding', 'wb' ) )

df_categoria_marca = pd.DataFrame(encoder.fit_transform(df5[['categoria_marca']]).toarray())
df_categoria_marca.columns = encoder.get_feature_names_out()
df5 = df5.join(df_categoria_marca)
# df5['categoria_marca'] = ohe.fit_transform(df5[['categoria_marca']])
pickle.dump( encoder, open( 'categoria_marca_encoding', 'wb' ) )

df_combustivel = pd.DataFrame(encoder.fit_transform(df5[['combustivel']]).toarray())
df_combustivel.columns = encoder.get_feature_names_out()
df5 = df5.join(df_combustivel)
# df5['combustivel'] = ohe.fit_transform(df5[['combustivel']])
df5 = df5.drop( columns=['cambio','tipo_vendedor','tipo_anuncio','categoria_marca','combustivel'] )
pickle.dump( encoder, open( 'combustivel_encoding', 'wb' ) )

# frequency encoder/label encoder: modelo, versao
# fe_modelo = df5.groupby('modelo').size() / len(df5)
# df5.loc[:,'modelo'] = df5['modelo'].map(fe_modelo)

# fe_versao = df5.groupby('versao').size() / len(df5)
# df5.loc[:,'versao'] = df5['versao'].map(fe_versao)
te = TargetEncoder()
# le = LabelEncoder()
# df5['modelo'] = le.fit_transform(df5['modelo'])
# # df5['modelo'] = te.fit_transform(df5['modelo'], df5['preco'])
# pickle.dump( le, open( 'modelo_encoding', 'wb' ) )

df5['versao'] = te.fit_transform(df5['versao'], df5['preco'])
pickle.dump( te, open( 'versao_encoding', 'wb' ) )
# df5['versao'] = le.fit_transform(df5['versao'])

# # fe_marca = df5.groupby('marca').size() / len(df5)
# # df5.loc[:,'marca'] = df5['marca'].map(fe_marca)

# # fe_cor = df5.groupby('cor').size() / len(df5)
# # df5.loc[:,'cor'] = df5['cor'].map(fe_cor)

# # fe_tipo = df5.groupby('tipo').size() / len(df5)
# # df5.loc[:,'tipo'] = df5['tipo'].map(fe_tipo)

# label encoder
# le = LabelEncoder()
df5['marca'] = te.fit_transform(df5['marca'],df5['preco'])
pickle.dump( te, open( 'marca_encoding', 'wb' ) )
df5['cor'] = te.fit_transform(df5['cor'],df5['preco'])
pickle.dump( te, open( 'cor_encoding', 'wb' ) )
df5['tipo'] = te.fit_transform(df5['tipo'],df5['preco'])
pickle.dump( te, open( 'tipo_encoding', 'wb' ) )
df5['cidade_vendedor'] = te.fit_transform(df5['cidade_vendedor'],df5['preco'])
pickle.dump( te, open( 'cidade_vendedor_encoding', 'wb' ) )
df5['estado_vendedor'] = te.fit_transform(df5['estado_vendedor'],df5['preco'])
pickle.dump( te, open( 'estado_vendedor_encoding', 'wb' ) )

# cols = ['marca','cor','tipo','cidade_vendedor','estado_vendedor','versao','modelo']
# ce = CountEncoder(cols=cols, return_df=True)
# df5[cols] = te.fit_transform(df5[cols], df5['preco'])
# pickle.dump( ce, open( 'ce_encoder', 'wb' ) )

df5['modelo'] = te.fit_transform(df5['modelo'], df5['preco'])
pickle.dump( te, open( 'modelo_encoding_te', 'wb' ) )

f:\SamuelOliveiraAlvesd\Anaconda3\envs\Health_Insurance_Cross_Sell\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
f:\SamuelOliveiraAlvesd\Anaconda3\envs\Health_Insurance_Cross_Sell\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


## 5.2. Normalização

## 5.3. Rescaling

In [21]:
# mms = MinMaxScaler()

# df5['num_fotos'] = mms.fit_transform( df5[['num_fotos']].values )
# # df5['ano_modelo'] = mms.fit_transform( df5[['ano_modelo']].values )
# df5['odometro'] = mms.fit_transform( df5[['odometro']].values )
# df5['num_portas'] = mms.fit_transform( df5[['num_portas']].values )

## 5.4. Transformação

In [22]:
df5['preco'] = np.log1p(df5['preco'])

# 6.0. FEATURE SELECTION

In [23]:
df6 = df5.copy()

## 6.1. Manual Selection

## 6.2. Feature Importancia

In [24]:
# # model
# forest = en.ExtraTreesRegressor( n_estimators=250, random_state=0, n_jobs=-1 )

# # training
# x_train_fselection = df6.drop( ['preco'], axis=1 )
# y_train_fselection = df6['preco'].values
# forest.fit( x_train_fselection, y_train_fselection )

In [25]:
# importances = forest.feature_importances_
# std = np.std( [tree.feature_importances_ for tree in forest.estimators_], axis=0 )
# indices = np.argsort( importances )[::-1]

# # print the feature ranking
# df = pd.DataFrame()

# print( 'Feature Ranking:\n' )
# for i, j in zip( x_train_fselection,forest.feature_importances_ ):
#     aux = pd.DataFrame( {'feature': i, 'importance': j}, index=[0] )
#     df = pd.concat( [df, aux], axis=0 )
    
# print( df.sort_values( 'importance', ascending=False ) ) 

# # plot the impurity-based feature importances of the forest
# plt.figure()
# plt.title( 'Feature importances' )
# plt.bar( range( x_train_fselection.shape[1] ), importances[indices], color='r', yerr=std[indices], align='center' )
# plt.xticks( range(x_train_fselection.shape[1]), indices )
# plt.xlim( [-1, x_train_fselection.shape[1]] )
# plt.show()

In [26]:
# a = df.sort_values( by='importance', ascending=False )

## 6.3. Boruta Selection

In [27]:
from boruta                import BorutaPy

In [29]:
# training
x_train_fselection = df6.drop( ['preco'], axis=1 ).values
y_train_fselection = df6['preco'].values.ravel()

# define RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1)

# define Boruta
boruta = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=42).fit(x_train_fselection, y_train_fselection)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	47
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	47
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	47
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	47
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	47
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	47
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	47
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	4
Rejected: 	43
Iteration: 	9 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	43
Iteration: 	10 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	43
Iteration: 	11 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	43
Iteration: 	12 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	43
Iteration: 	13 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	43
Iteration: 	14 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	43
Iteration: 	15 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	43
Iteration: 	16 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	43
I

In [31]:
cols_selected = boruta.support_.tolist()

# best features
X_train_fs = df6.drop(['preco'], axis=1)
cols_selected_boruta = X_train_fs.iloc[:, cols_selected]

# not selected boruta
cols_not_selected_boruta = list(np.setdiff1d(X_train_fs.columns, cols_selected_boruta.columns))

cols_selected_boruta.columns

Index(['modelo', 'versao', 'odometro'], dtype='object')

In [32]:
cols_not_selected_boruta

['Year_Bin1',
 'Year_Bin2',
 'Year_Bin3',
 'Year_Bin4',
 'Year_Bin5',
 'Year_Bin6',
 'ano_modelo',
 'blindado',
 'cambio_Automatizada',
 'cambio_Automatizada DCT',
 'cambio_Automática',
 'cambio_Automática Sequencial',
 'cambio_CVT',
 'cambio_Manual',
 'cambio_Semi-automática',
 'categoria_marca_luxo',
 'categoria_marca_popular_alto_padrao',
 'categoria_marca_popular_baixo_padrao',
 'categoria_marca_superluxo',
 'cidade_vendedor',
 'cilindrada',
 'combustivel_DESCONHECIDO',
 'combustivel_DIESEL',
 'combustivel_ELÉTRICO',
 'combustivel_FLEX',
 'combustivel_GASOLINA',
 'combustivel_HYBRID',
 'combustivel_HÍBRIDO',
 'cor',
 'entrega_delivery',
 'estado_vendedor',
 'marca',
 'num_fotos',
 'num_portas',
 'offroad',
 'tipo',
 'tipo_anuncio_Acessórios e serviços para autos',
 'tipo_anuncio_Concessionária',
 'tipo_anuncio_Loja',
 'tipo_anuncio_Pessoa Física',
 'tipo_vendedor_PF',
 'tipo_vendedor_PJ',
 'troca',
 'turbo']

In [ ]:
# columns = ['ano_modelo','odometro','cambio_Manual','versao','cor','tipo','cidade_vendedor','modelo','estado_vendedor','num_fotos','marca',
# 'combustivel_DIESEL','combustivel_FLEX','combustivel_GASOLINA','combustivel_DESCONHECIDO','combustivel_HYBRID','combustivel_HÍBRIDO',
# 'categoria_marca_popular_alto_padrao','categoria_marca_superluxo','categoria_marca_popular_baixo_padrao','categoria_marca_luxo','Year_Bin6','Year_Bin5','Year_Bin4','Year_Bin3','Year_Bin2',
# 'Year_Bin1','Year_Bin6']

# 7.0. MACHINE LEARNING MODELLING

In [ ]:
df7 = df6.copy()

X = df7.drop(columns=["preco"])
# df7 = df7[columns]
Y = df7['preco'].copy()
# Y = df6['preco'].copy()

X_train, X_val, y_train, y_val = ms.train_test_split( X, Y, test_size=0.2, random_state=42 )

## 7.1. Linear Regression

In [ ]:
# model training
lr = LinearRegression().fit( X_train, y_train )

# prediction
yhat_lr = lr.predict( X_val )

# performance (error)
lr_results = ml_error( 'Linear Regression', np.expm1( y_val ) , np.expm1( yhat_lr ) )
lr_results

,Model name,MAE,MAPE,RMSE
0,Linear Regression,29588.931281,0.229434,54775.972229


### 7.1.1. Cross Validation

## 7.3. Random Forest Regressor

In [ ]:
# model
rf = RandomForestRegressor( n_estimators=150, n_jobs=-1, random_state=42 ).fit( X_train, y_train )

# prediction
yhat_rf = rf.predict( X_val )

# performance
rf_results = ml_error( 'Random Forest Regressor', np.expm1(y_val), np.expm1(yhat_rf) )
rf_results

,Model name,MAE,MAPE,RMSE
0,Random Forest Regressor,25722.723763,0.202168,42935.860987


### 7.3.1. Cross Validation

## 7.5. XGBoost Regressor

In [ ]:
# model
model_xgb = xgb.XGBRegressor( n_estimators=400 ).fit( X_train, y_train )

# prediction
yhat_xgb = model_xgb.predict( X_val )

# performance
model_xgb_results = ml_error( 'XGBoost Regressor',  np.expm1(y_val), np.expm1(yhat_xgb) )
model_xgb_results

C:\Users\mathe\anaconda3\envs\ds_em_producao\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


,Model name,MAE,MAPE,RMSE
0,XGBoost Regressor,25937.086046,0.203474,43418.215059


### 7.5.1. Cross Validation

## 7.6. LightGBM

In [ ]:
# model
model_lgb = lgb.LGBMRegressor(n_jobs=-1, random_state=42, subsample_freq=1, max_bin=500,
                              n_estimators=1181, max_depth=10, learning_rate=0.01027, 
                              num_leaves=128, min_child_samples=1, subsample=0.92676, 
                              colsample_bytree=0.68369).fit( X_train, y_train )

# prediction
yhat_lgb = model_lgb.predict( X_val )

# performance
model_lgb_results = ml_error( 'LightGBM Regressor',  np.expm1(y_val), np.expm1(yhat_lgb) )
model_lgb_results

,Model name,MAE,MAPE,RMSE
0,LightGBM Regressor,24882.546284,0.194521,41827.41902


In [ ]:
X_train.columns

Index(['num_fotos', 'marca', 'modelo', 'versao', 'ano_modelo', 'odometro',
       'num_portas', 'tipo', 'blindado', 'cor', 'cidade_vendedor',
       'estado_vendedor', 'entrega_delivery', 'troca', 'Year_Bin1',
       'Year_Bin2', 'Year_Bin3', 'Year_Bin4', 'Year_Bin5', 'Year_Bin6',
       'cilindrada', 'turbo', 'offroad', 'cambio_Automatizada',
       'cambio_Automatizada DCT', 'cambio_Automática',
       'cambio_Automática Sequencial', 'cambio_CVT', 'cambio_Manual',
       'cambio_Semi-automática', 'tipo_vendedor_PF', 'tipo_vendedor_PJ',
       'tipo_anuncio_Acessórios e serviços para autos',
       'tipo_anuncio_Concessionária', 'tipo_anuncio_Loja',
       'tipo_anuncio_Pessoa Física', 'categoria_marca_luxo',
       'categoria_marca_popular_alto_padrao',
       'categoria_marca_popular_baixo_padrao', 'categoria_marca_superluxo',
       'combustivel_DESCONHECIDO', 'combustivel_DIESEL',
       'combustivel_ELÉTRICO', 'combustivel_FLEX', 'combustivel_GASOLINA',
       'combustivel_HYBRID

In [ ]:
# join dfs
Full_X_train = pd.concat( [X_train, X_val], axis=0 )
Full_Y_train = pd.concat( [y_train, y_val], axis=0 )

# model
model_lgb_full = lgb.LGBMRegressor( n_jobs=-1, random_state=42, subsample_freq=1, max_bin=500,
                              n_estimators=1181, max_depth=10, learning_rate=0.01027, 
                              num_leaves=128, min_child_samples=1, subsample=0.92676, 
                              colsample_bytree=0.68369 ).fit( Full_X_train, Full_Y_train )

# saving trained model
pickle.dump( model_lgb_full, open( '/Users/mathe/Repos_ComunidadeDS/mobility_cars_hackday/mobility_cars_lgb.pkl', 'wb' ) )